In [159]:
import pandas as pd
import numpy as np


In [160]:
merged = pd.read_csv("/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/02_Cleaned_Data/merged.csv")
risk = pd.read_csv("/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/Outputs/risk_predictions.csv")
outliers = pd.read_csv("/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/Outputs/high_friction_outliers.csv")


In [ ]:

print("Merged rows:", len(merged))
print("Risk rows:", len(risk))


Merged rows: 4398
Risk rows: 880


In [162]:
# Reset indices to guarantee alignment
merged_reset = merged.reset_index(drop=True)
risk_reset = risk.reset_index(drop=True)


# PRIVACY-PRESERVING AGE VERIFICATION

In [ ]:
# Create age verification table
age_verification_table = merged_reset.copy()
age_verification_table['risk_score'] = risk_reset['risk_score']


In [ ]:
# Age Verification

def verify_age_privately(row, stress_threshold=0.85):

    # If system is under high predicted stress, defer verification
    if row['risk_score'] >= stress_threshold:
        return {
            "eligible": None,
            "status": "deferred_due_to_system_stress"
        }

    # Otherwise return binary eligibility only
    if row['age_18_greater'] > 0:
        return {
            "eligible": True,
            "status": "verified"
        }
    else:
        return {
            "eligible": False,
            "status": "verified"
        }



In [ ]:
# Apply age verification function
age_verification_table['age_verification_result'] = (
    age_verification_table.apply(verify_age_privately, axis=1)
)


In [166]:
age_verification_table[
    ['state', 'district', 'month', 'risk_score', 'age_verification_result']
]


,state,district,month,risk_score,age_verification_result
0,Andaman and Nicobar Islands,nicobar,9,0.007412,"{'eligible': False, 'status': 'verified'}"
1,Andaman and Nicobar Islands,nicobar,10,0.001601,"{'eligible': False, 'status': 'verified'}"
2,Andaman and Nicobar Islands,nicobar,11,0.025498,"{'eligible': False, 'status': 'verified'}"
3,Andaman and Nicobar Islands,nicobar,12,0.000987,"{'eligible': False, 'status': 'verified'}"
4,Andaman and Nicobar Islands,north and middle andaman,9,0.858008,"{'eligible': None, 'status': 'deferred_due_to_..."
...,...,...,...,...,...
4393,West Bengal,uttar dinajpur,12,NaN,"{'eligible': True, 'status': 'verified'}"
4394,West Bengal,west midnapore,9,NaN,"{'eligible': False, 'status': 'verified'}"
4395,West Bengal,west midnapore,10,NaN,"{'eligible': False, 'status': 'verified'}"
4396,West Bengal,west midnapore,11,NaN,"{'eligible': False, 'status': 'verified'}"


In [ ]:
# Summary 
age_verification_table['age_verification_result'].apply(
    lambda x: x['status']
).value_counts()


age_verification_result
verified                         4372
deferred_due_to_system_stress      26
Name: count, dtype: int64

In [ ]:
# final output table
age_verification_output = age_verification_table.copy()

age_verification_output['eligible'] = (
    age_verification_output['age_verification_result']
    .apply(lambda x: x['eligible'])
)

age_verification_output['status'] = (
    age_verification_output['age_verification_result']
    .apply(lambda x: x['status'])
)

age_verification_output = age_verification_output[
    ['state', 'district', 'month', 'risk_score', 'eligible', 'status']
]


In [169]:
age_verification_output


,state,district,month,risk_score,eligible,status
0,Andaman and Nicobar Islands,nicobar,9,0.007412,False,verified
1,Andaman and Nicobar Islands,nicobar,10,0.001601,False,verified
2,Andaman and Nicobar Islands,nicobar,11,0.025498,False,verified
3,Andaman and Nicobar Islands,nicobar,12,0.000987,False,verified
4,Andaman and Nicobar Islands,north and middle andaman,9,0.858008,None,deferred_due_to_system_stress
...,...,...,...,...,...,...
4393,West Bengal,uttar dinajpur,12,NaN,True,verified
4394,West Bengal,west midnapore,9,NaN,False,verified
4395,West Bengal,west midnapore,10,NaN,False,verified
4396,West Bengal,west midnapore,11,NaN,False,verified


In [170]:
age_verification_output.to_csv(
    "/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/Outputs/age_verification_output.csv",
    index=False
)


# DISASTER RELIEF EARLY WARNING SYSTEM

In [171]:
# Reset indices to guarantee alignment
disaster_df = merged_reset.copy()
disaster_df['risk_score'] = risk_reset['risk_score']

In [172]:
# expected update behavior per district

baseline = (
    disaster_df
    .groupby(['state', 'district'], as_index=False)
    .agg(baseline_updates=('total_updates', 'mean'))
)



In [173]:
# Compare Current Month vs Baseline

disaster_df = disaster_df.merge(
    baseline,
    on=['state', 'district'],
    how='left',
    validate='many_to_one'  # important: sanity check
)


In [174]:
print("Columns after baseline merge:")
print(disaster_df.columns.tolist())


Columns after baseline merge:
['state', 'district', 'year', 'month', 'age_0_5', 'age_5_17', 'age_18_greater', 'demo_age_5_17', 'demo_age_17_', 'bio_age_5_17', 'bio_age_17_', 'total_enrolments', 'total_updates', 'adult_system_stress', 'child_system_stress', 'digital_friction_index', 'log_digital_friction', 'friction_outlier', 'risk_score', 'baseline_updates']


In [175]:
# update spike ratio calculation
disaster_df['update_spike_ratio'] = (
    disaster_df['total_updates'] /
    (disaster_df['baseline_updates'].fillna(0) + 1)
)


In [176]:
# Disaster Alert Logic

def disaster_alert(row):
    # Severe displacement + system stress
    if row['update_spike_ratio'] >= 1.8 and row['risk_score'] >= 0.7:
        return "HIGH_ALERT"

    # Early warning (most common)
    elif row['update_spike_ratio'] >= 1.5:
        return "WATCH"

    else:
        return "NORMAL"
    




In [177]:
disaster_df['disaster_alert'] = disaster_df.apply(disaster_alert, axis=1)

In [178]:
disaster_df['disaster_alert'].value_counts()


disaster_alert
NORMAL    4070
WATCH      328
Name: count, dtype: int64

In [179]:
simulated = disaster_df.copy()
simulated.loc[
    simulated['update_spike_ratio'] > 1.6,
    'update_spike_ratio'
] *= 1.3

simulated['disaster_alert_simulated'] = simulated.apply(disaster_alert, axis=1)

simulated['disaster_alert_simulated'].value_counts()


disaster_alert_simulated
NORMAL    4070
WATCH      328
Name: count, dtype: int64

In [180]:
# Actionable Alert Table
disaster_alerts = disaster_df[
    disaster_df['disaster_alert'] != "NORMAL"
][
    [
        'state',
        'district',
        'month',
        'total_updates',
        'baseline_updates',
        'update_spike_ratio',
        'risk_score',
        'disaster_alert'
    ]
]


In [ ]:

disaster_alerts.sort_values(
    ['disaster_alert', 'update_spike_ratio'],
    ascending=[True, False]
)

,state,district,month,total_updates,baseline_updates,update_spike_ratio,risk_score,disaster_alert
2540,Manipur,imphal west,11,38933.0,13491.600000,2.885508,NaN,WATCH
4181,Uttarakhand,bijnor,12,1078.0,377.500000,2.848085,NaN,WATCH
487,Assam,karimganj,12,20642.0,7585.125000,2.721020,0.004149,WATCH
2566,Meghalaya,east garo hills,9,1676.0,630.375000,2.654524,NaN,WATCH
2641,Meghalaya,west jaintia hills,3,3241.0,1254.777778,2.580871,NaN,WATCH
...,...,...,...,...,...,...,...,...
3010,Punjab,ludhiana,3,46575.0,30970.444444,1.503805,NaN,WATCH
2454,Maharashtra,raigad,11,10168.0,6767.250000,1.502309,NaN,WATCH
431,Assam,golaghat,11,6053.0,4028.250000,1.502265,0.000259,WATCH
2042,Madhya Pradesh,burhanpur,6,14379.0,9575.125000,1.501547,NaN,WATCH


In [ ]:
# Final output table
disaster_df['disaster_alert'] = disaster_df.apply(disaster_alert, axis=1)

disaster_df[['state', 'district', 'month', 'risk_score', 'update_spike_ratio', 'disaster_alert']]

,state,district,month,risk_score,update_spike_ratio,disaster_alert
0,Andaman and Nicobar Islands,nicobar,9,0.007412,1.352219,NORMAL
1,Andaman and Nicobar Islands,nicobar,10,0.001601,0.487252,NORMAL
2,Andaman and Nicobar Islands,nicobar,11,0.025498,0.966950,NORMAL
3,Andaman and Nicobar Islands,nicobar,12,0.000987,1.178470,NORMAL
4,Andaman and Nicobar Islands,north and middle andaman,9,0.858008,1.192959,NORMAL
...,...,...,...,...,...,...
4393,West Bengal,uttar dinajpur,12,NaN,0.949020,NORMAL
4394,West Bengal,west midnapore,9,NaN,1.141422,NORMAL
4395,West Bengal,west midnapore,10,NaN,0.822103,NORMAL
4396,West Bengal,west midnapore,11,NaN,1.212187,NORMAL


In [183]:
disaster_df[['state', 'district', 'month', 'risk_score', 'update_spike_ratio', 'disaster_alert']].to_csv(
    "/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/Outputs/disaster_early_alerts.csv",
    index=False
)


# AADHAAR KENDRA OPERATIONS & STAFFING OPTIMIZATION

In [184]:
ops_df = merged_reset.copy()
ops_df['risk_score'] = risk_reset['risk_score']

In [185]:
# Identify Dominant Stress Type

def dominant_stress(row):
    if row['adult_system_stress'] > row['child_system_stress']:
        return "ADULT"
    elif row['child_system_stress'] > row['adult_system_stress']:
        return "CHILD"
    else:
        return "BALANCED"


In [186]:
ops_df['dominant_stress'] = ops_df.apply(dominant_stress, axis=1)


In [187]:
ops_df[['state', 'district', 'month', 'risk_score', 'dominant_stress']]

,state,district,month,risk_score,dominant_stress
0,Andaman and Nicobar Islands,nicobar,9,0.007412,ADULT
1,Andaman and Nicobar Islands,nicobar,10,0.001601,ADULT
2,Andaman and Nicobar Islands,nicobar,11,0.025498,ADULT
3,Andaman and Nicobar Islands,nicobar,12,0.000987,CHILD
4,Andaman and Nicobar Islands,north and middle andaman,9,0.858008,CHILD
...,...,...,...,...,...
4393,West Bengal,uttar dinajpur,12,NaN,ADULT
4394,West Bengal,west midnapore,9,NaN,ADULT
4395,West Bengal,west midnapore,10,NaN,ADULT
4396,West Bengal,west midnapore,11,NaN,ADULT


In [188]:
# Kendra Optimization Rules

def kendra_recommendation(row):
    # Critical overload
    if row['risk_score'] >= 0.85:
        if row['dominant_stress'] == "ADULT":
            return "Add adult biometric operators + extend hours"
        elif row['dominant_stress'] == "CHILD":
            return "Schedule school-hour update camps"
        else:
            return "Deploy mobile enrolment unit"

    # High but manageable stress
    elif row['risk_score'] >= 0.65:
        return "Increase counter capacity during peak hours"

    # Mild stress
    elif row['risk_score'] >= 0.45:
        return "Citizen advisory: visit during off-peak hours"

    # Normal operations
    else:
        return "Normal operations"


In [ ]:
ops_df['kendra_recommendation'] = ops_df.apply(
    kendra_recommendation,
    axis=1
)

In [190]:
ops_df[['state', 'district', 'month', 'risk_score', 'dominant_stress', 'kendra_recommendation']]

,state,district,month,risk_score,dominant_stress,kendra_recommendation
0,Andaman and Nicobar Islands,nicobar,9,0.007412,ADULT,Normal operations
1,Andaman and Nicobar Islands,nicobar,10,0.001601,ADULT,Normal operations
2,Andaman and Nicobar Islands,nicobar,11,0.025498,ADULT,Normal operations
3,Andaman and Nicobar Islands,nicobar,12,0.000987,CHILD,Normal operations
4,Andaman and Nicobar Islands,north and middle andaman,9,0.858008,CHILD,Schedule school-hour update camps
...,...,...,...,...,...,...
4393,West Bengal,uttar dinajpur,12,NaN,ADULT,Normal operations
4394,West Bengal,west midnapore,9,NaN,ADULT,Normal operations
4395,West Bengal,west midnapore,10,NaN,ADULT,Normal operations
4396,West Bengal,west midnapore,11,NaN,ADULT,Normal operations


In [191]:
# Inspect Recommendation Distribution

ops_df['kendra_recommendation'].value_counts()


kendra_recommendation
Normal operations                                4342
Add adult biometric operators + extend hours       18
Citizen advisory: visit during off-peak hours      16
Increase counter capacity during peak hours        14
Schedule school-hour update camps                   5
Deploy mobile enrolment unit                        3
Name: count, dtype: int64

In [192]:
# Actionable Operations Table

ops_actions = ops_df[
    ops_df['kendra_recommendation'] != "Normal operations"
][[
    'state',
    'district',
    'month',
    'risk_score',
    'dominant_stress',
    'kendra_recommendation'
]]


In [193]:
ops_actions.sort_values(
    'risk_score',
    ascending=False
)

,state,district,month,risk_score,dominant_stress,kendra_recommendation
214,Andhra Pradesh,vikarabad,9,0.983939,ADULT,Add adult biometric operators + extend hours
570,Assam,south salmara mankachar,11,0.980257,ADULT,Add adult biometric operators + extend hours
152,Andhra Pradesh,narayanpet,10,0.979604,ADULT,Add adult biometric operators + extend hours
242,Andhra Pradesh,west godavari,11,0.976174,ADULT,Add adult biometric operators + extend hours
425,Assam,golaghat,4,0.974702,CHILD,Schedule school-hour update camps
328,Arunachal Pradesh,west kameng,10,0.972611,ADULT,Add adult biometric operators + extend hours
528,Assam,nalbari,5,0.971945,CHILD,Schedule school-hour update camps
237,Andhra Pradesh,warangal rural,11,0.964071,BALANCED,Deploy mobile enrolment unit
123,Andhra Pradesh,mahabubnagar,10,0.960151,ADULT,Add adult biometric operators + extend hours
396,Assam,dhemaji,9,0.957097,ADULT,Add adult biometric operators + extend hours


In [194]:
ops_actions.to_csv(
    "/Users/ayush/Library/Mobile Documents/com~apple~CloudDocs/Developer/UIDAI/Outputs/kendra_optimization_recommendations.csv",
    index=False
)
